In [1]:
import torch
import torchvision
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import onnx

In [2]:
model = ssd300_vgg16(pretrained=True)
model.eval()

NameError: name 'ssd300_vgg16' is not defined

In [ ]:
COCO_CLASSES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
    'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', ...
]


In [ ]:
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose({
        transforms.Resize(300),
        transforms.ToTensor(),
    })
    image_tensor = transform(image).unsqueeze(0)
    return image, image_tensor

In [ ]:
# running inference and drawing bounding boxes
def detect_objects(image_path):
    image, image_tensor = preprocess_image(image_path)
    with torch.no_grad():
        predictions = model(image_tensor)
    # extract bounding boxes, labels, and scores
    boxes = predictions[0]['boxes']
    labels = predictions[0]['labels']
    scores = predictions[0]['scores']

    draw = ImageDraw.Draw(image)

    for box, label, score in zip(boxes, labels, scores):
        if score > 0.5:
            x1, y1, x2, y2 = map(int, box)
            draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
            draw.text((x1, y1 - 10), f"{COCO_CLASSES[label]}: {score:.2f}", fill="black")
    image.show()

Let's attempt to identify Ms Kasey on the Severed floor of Lumen.

![mskasey](mskasey.png)

In [ ]:
detect_objects("mskasey.png")

Here is our bounding box with person labelled

![mskasey found](mskasey_found.jpg)

save model to onnx format

In [ ]:
model = torchvision.models.detection.ssd300_vgg16(pretrained=True)
model.eval()

dummy_input = torch.randn(1, 3, 300, 300)

torch.onnx.export(model, dummy_input, "mobilenetv2_ssd.onnx",
    opset_version=11,
    input_names=["input"],
    output_names=["boxes", "labels", "scores"]
)

/Users/jakehenderson/nocuments/code/projects/ml-daily/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/jakehenderson/nocuments/code/projects/ml-daily/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SSD300_VGG16_Weights.COCO_V1`. You can also use `weights=SSD300_VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/Users/jakehenderson/nocuments/code/projects/ml-daily/.venv/lib/python3.13/site-packages/torchvision/ops/boxes.py:166: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), ra

In [5]:
import os

onnx_model_path = "mobilenetv2_ssd.onnx"
# report file size in MB
file_size = os.path.getsize(onnx_model_path) / (1024 * 1024)
print(f"ONNX model size (MB): {file_size:.2f}")

ONNX model size (MB): 136.28


In [6]:
#check ram usage when loaded in ONNX runtime
import onnxruntime
import psutil

In [8]:
session = onnxruntime.InferenceSession(onnx_model_path)

#get memory usage
process = psutil.Process()
memory_usage = process.memory_info().rss / (1024 * 1024)
print(f"RAM Usage after loading: {memory_usage:.2f} MB")

RAM Usage after loading: 737.55 MB
